# GPU-Accelerated Gauss-Seidel Sparse Iterative Solver using Graph Coloring

### Load Suite Sparse matrix for testing
Use ssget and search for 'filter2D', a symmetric matrix used in structural problems.

In [ ]:
!pip install ssgetpy --quiet
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git --quiet
%load_ext nvcc_plugin

  Preparing metadata (setup.py) ... done
created output directory at /content/src
Out bin /content/result.out


In [ ]:
import ssgetpy
ssgetpy.search(name="filter2D") # Ideal matrix after searching properties

1430,Oberwolfach,filter2D,1668,1668,10750,real,Yes,No,1.0,1.0,model reduction problem,


In [ ]:
import ssgetpy
result = ssgetpy.search(name="flowmeter0")[0]
result.download(format='MM', destpath='content/', extract=True)

flowmeter0:   0%|          | 0/487663 [00:00<?, ?B/s]

('content/flowmeter0', 'content/flowmeter0.tar.gz')

### CPU Gauss-Seidel

This approach is given in a gist by Eric Arneback at https://gist.github.com/Erkaman/b34b3531e209a1db38e259ea53ff0be9. The graph coloring is not used in this purely sequential approach, and the following cell considers each element of the *x* vector to updated one by one from start to end.

This sequential version of Gauss-Seidel serves as a base case, and it is used for testing the convergence of Sequential Gauss Seidel on the Suite Sparse matrix.

In [ ]:
%%writefile cpu.cpp

#include <stdio.h>
#include <math.h>
#include <stdlib.h>
#include <chrono>
#include <iostream>
#include <set>
#include <vector>
#include <fstream>
#include <algorithm>

const float EPS = 0.00001f;
typedef std::vector<int> Partition;

// vector of dimension Nx1
class Vec {
public:
  int N;
	float* v;

	Vec(int n) {
    N = n;
    v = new float[N];
		for (int i = 0; i < N; ++i) {
			v[i] = 0.0f;
		}
	}

	~Vec()
	{
		delete[] v;
	}

	void print() {
		for (int i = 0; i < N; ++i) {
			printf("%f, ", v[i]);
		}
		printf("\n");
	}
};

// matrix of dimension NxN.
class Mat {
public:
  int N;
	float* m;

	Mat(int n) {
    N = n;
    m = new float[N*N];
		for (int i = 0; i < N*N; ++i) {
			m[i] = 0.0f;
		}
	}

	~Mat()
	{
		delete[] m;
	}

	void print() {
		for (int i = 0; i < N*N; ++i) {
			printf("%f, ", m[i]);
			if (i % N == 0){printf("\n");}
		}
		printf("\n");
	}

	Vec* mult(const Vec* v) const {
		Vec* r = new Vec(N);

		for (int row = 0; row < N; ++row) {
			for (int col = 0; col < N; ++col) {
				r->v[row] += this->m[row*N+col] * v->v[col];
			}
		}
		return r;
	}
};


int gauss_seidel(Vec* x, const Vec* b, const Mat* m, float tol, int maxiter, int N) {

	int iter;
	float old_norm = 0.0f;
	for (iter = 0; iter < maxiter; ++iter) {
		for (int i = 0; i < N; i++) {
				float s = 0.0f;
				float c = 0.0f;
				for (int j = 0; j < N; ++j) {
					if (j != i) {
						//s += m->m[i*N+j] * x->v[j];
						float y = (m->m[i*N+j] * x->v[j]) - c;
						float t = s + y;
						c = (t - s) - y;
						s = t;
					}
				}
				x->v[i] = (1.0f / m->m[i*N+i]) * (b->v[i] - s);
		}

		Vec* mx = new Vec(N);
		mx = m->mult(x);

		float norm = 0.0f;
		float c = 0.0f;
		for (int i = 0; i < N; ++i) {
			float a = mx->v[i] - b->v[i];
			//norm += a*a;
			float y = a*a - c;
			float t = norm + y;
			c = (t-norm) - y;
			norm = t;
		}
		norm = sqrt(norm);
		if (fabs(norm-old_norm) < tol) {
			break;
		}
		old_norm = norm;
		delete mx;
	}

	return iter;
}


int main() {
	srand(13000);

  std::ifstream file("content/flowmeter0/flowmeter0.mtx");
  int num_row, num_col, num_lines, N;

  while (file.peek() == '%') file.ignore(2048, '\n');

  file >> num_row>> num_col >> num_lines;

  N = num_row;

	Mat* m = new Mat(N);
  std::fill(m->m, m->m + num_row * num_col, 0.0f);
  for (int l = 0; l < num_lines; l++)
  {
      float data;
      int row, col;
      file >> row >> col >> data;
      m->m[(row -1) + (col -1) * num_row] = data;
  }

  file.close();

	int nonZeros = 0;
	for (int i = 0; i < N; ++i) {
		for (int j = i; j < N; ++j) {
			if (fabs(m->m[i*N+j]) > EPS) {
				nonZeros++;
			}
		}
	}
	printf("Normal Gauss Seidel \npercent of non-zeros of M: %d%%\n", int(100.0f * float(nonZeros) / float(N*N)));

	Vec* expected_solution = new Vec(N);
	for (int i = 0; i < N; ++i) {
		expected_solution->v[i] = 8.0f * float(rand() % 100) / 100.0f - 4.0f;
	}

	Vec* b = new Vec(N);
	b = m->mult(expected_solution);

	/*
	With that, we have generated a linear system
	M*x = b.
	Now let's solve it!
	*/

	Vec* x = new Vec(N);
	for (int i = 0; i < N; ++i) {
		x->v[i] = 0.0f;
	}

	printf("solving linear system where N = %d", N);

	printf("\n");

	auto started = std::chrono::high_resolution_clock::now();
	int iter = gauss_seidel(x, b, m, 0.001f, 10000, N);
	auto done = std::chrono::high_resolution_clock::now();

	std::cout << "Gauss Seidel CPU time: " << std::chrono::duration_cast<std::chrono::milliseconds>(done - started).count() << "ms\n";
	printf("number of iterations: %d\n", iter);

	delete m;
	delete x;
	delete b;
	delete expected_solution;
}

Overwriting cpu.cpp


In [ ]:
!g++ cpu.cpp -o cpu
!./cpu

Normal Gauss Seidel 
percent of non-zeros of M: 0%
solving linear system where N = 9669
Gauss Seidel CPU time: 30396ms
number of iterations: 20


### Comparing Brooks Vizing coloring with CSRColor

Using the CPU graph coloring algorthm (Brooks Vizing) that was also used by Vivace and provided by Eric Arneback, we observe the performance of the code, as well as the quality of the result, based on the number of colors received. Having fewer colors is ideal because there will be fewer kernel calls per iteration and a significant speedup in the Gauss-Seidel Kernel. However, the time taken by the graph coloring should also be small enough for real-time.

In [ ]:
%%writefile cpucoloringgpugs.cu

#include <stdio.h>
#include <math.h>
#include <stdlib.h>
#include <chrono>
#include <iostream>
#include <set>
#include <vector>
#include <fstream>
#include <algorithm>

const float SHRINKING_FACTOR = 7.5f;
const int NO_PROGRESS_STREAK_THRESHOLD = 100;
const float EPS = 0.00001f;
typedef std::vector<int> Partition;

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
	if (err != cudaSuccess) {
		fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
	}
	return err;
}

// vector of dimension Nx1
template <class T>
class gpuVec {
public:
	T* v;
  int N;

	__host__ gpuVec(int size) {
    N = size;
		cudaMallocManaged(&v, (T)(size * (int)sizeof(int)));
		for (int i = 0; i < N; ++i) {
			v[i] = 0;
		}
	}

	__host__ ~gpuVec()
	{
		cudaFree(v);
	}

	__host__ void print() {
		for (int i = 0; i < N; ++i) {
			printf("%f, ", v[i]);
		}
		printf("\n");
	}
};

// matrix of dimension NxN.
class gpuMat {
public:
	float* m;
  int N;

	__host__ gpuMat(int n) {
    N = n;
		cudaMallocManaged(&m, (float)(N * N * (int)sizeof(float)));
		for (int i = 0; i < N * N; ++i) {
			m[i] = 0.0f;
		}
	}

	__host__ ~gpuMat()
	{
		cudaFree(m);
	}

	__host__ void printdiag() {
		for (int i = 0; i < N; ++i) {
			{ printf("%f, ", m[i * N + i]); }
		}
		printf("\n");
	}

	__host__ void print() {
		for (int i = 0; i < N * N; ++i) {
			printf("%f, ", m[i]);
			if (i % N == 0 && i != 0) { printf("\n"); }
		}
		printf("\n");
	}

  // only used to get solution, b, from initial matrix, m.
  __host__ gpuVec<float>* gpu_mult(const gpuVec<float>* v, gpuVec<float>* r) const {

		for (int row = 0; row < N; ++row) {
			for (int col = 0; col < N; ++col) {
				r->v[row] += this->m[row * N + col] * v->v[col];
			}
		}
		return r;
	}
};

std::vector<int> randomized_graph_coloring(gpuMat* m, int* colors, int N) {
	std::set<int> neighbours[N];

	int node_colors[N]; // colors assigned to the nodes.
	int next_color[N]; // next color of every node, in case the palette runs out.
	std::set<int> node_palettes[N]; // palettes of the nodes.
	std::set<int> U;

	/*
	Every node needs to know about it's neighbours. so find that.
	There is an edge between two nodes i and j, if the matrix coefficient at row i, column j is non-zero.
	*/
	for (int i = 0; i < N; ++i) {
		for (int j = 0; j < N; ++j) {
			if (i != j && fabs(m->m[i * N + j]) > EPS) {

				// if necessary, make j a neighbour of i.
				if (neighbours[i].find(j) == neighbours[i].end()) {
					neighbours[i].insert(j);
				}

				// if necessary, make i a neighbour of j.
				if (neighbours[j].find(i) == neighbours[j].end()) {
					neighbours[j].insert(i);
				}
			}
		}
	}

	// calculate max degree of a single node.
	int delta_v = 0;
	for (int i = 0; i < N; ++i) {
		if ((int)neighbours[i].size() > delta_v) {
			delta_v = neighbours[i].size();
		}
	}

	// initially, every node has a palette of size delta_v/shrinking_factor.
	// the maximum number of colors necessary for a graph coloring is delta_v, but many
	// graphs won't need that many colors. therefore, we choose to shrink delta_v by a shrinking factor.
	// if the shrinking factor is too big, so that the problem is unsolvable, then more colors will be added on the fly.
	int max_color = int((float)delta_v / SHRINKING_FACTOR);
	if (max_color <= 0) {
		max_color = 1;
	}
	//max_color = 2;

	// initialize the palettes for all the node.
	// the colors in the palette will be chosen randomly from, for all the remaining nodes in U.
	for (int iv = 0; iv < N; ++iv) {
		for (int ic = 0; ic < max_color; ++ic) {
			node_palettes[iv].insert(ic);
		}
		next_color[iv] = max_color;
	}

	for (int iv = 0; iv < N; ++iv) {
		U.insert(iv);
	}

	// keep track of the number of iterations with no progress.
	int no_progress_streak = 0;

	/*
	If a node has found a color that solves the graph coloring for that node, then remove from U.
	Once U is empty, the graph coloring problem is done.
	*/
	while (U.size()) {

		// all remaining nodes in U are given a random color.
		for (int iv : U) {
			// get random color from palette, and assign it.
			int m = rand() % node_palettes[iv].size();
			auto setIt = node_palettes[iv].begin();
			advance(setIt, m);

			node_colors[iv] = *setIt;
		}

		std::set<int> temp;


		/*
		  Now let's find all the nodes whose colors are different from all their neighbours.
		  Those nodes will be removed from U, because they are done, with respect to the graph coloring problem.
		*/
		for (int iv : U) {

			int icolor = node_colors[iv];

			/*
			Check if graph coloring property is solved for node.
			*/
			bool different_from_neighbours = true;
			for (int neighbour : neighbours[iv]) {

				if (node_colors[neighbour] == icolor) {
					different_from_neighbours = false;
					break;
				}
			}

			if (different_from_neighbours) {
				// found the right color for this one.
				// so remove from U.

				// also, the neighbours of iv can't use this color anymore.
				// so remove it from their palettes.
				for (int neighbour : neighbours[iv]) {
					node_palettes[neighbour].erase(icolor);
				}

			}
			else {
				// not a correct color. don't remove from U.
				temp.insert(iv);
			}

			// feed the hungry!
			// if palette empty, we add more colors on the fly.
			// if we don't do this, the algorithm will get stuck in a loop.
			if (node_palettes[iv].empty()) {
				node_palettes[iv].insert(next_color[iv]++);
			}

		}

		if (U.size() == temp.size()) {
			no_progress_streak++;

			// if no progress for too many iterations, we have no choice but to feed a random node.
			if (no_progress_streak > NO_PROGRESS_STREAK_THRESHOLD) {
				int m = rand() % U.size();
				auto setIt = U.begin();
				advance(setIt, m);

				node_palettes[*setIt].insert(next_color[*setIt]++);

				no_progress_streak = 0;
			}
		}

		U = temp;
	}

	// find the number of colors used in our solution.
	// this is also the number of partitions.
	int num_colors = 0;
	for (int i = 0; i < N; ++i) {
		if (next_color[i] > num_colors) {
			num_colors = next_color[i];
		}
	}

	/*
	Finally, we collect all the partitions then.
	*/
	std::vector<Partition> partitions;
	for (int ic = 0; ic < num_colors; ++ic) {
		Partition partition;

		/*
		The first partition is all nodes that use color 0,
		the second partition use color 1, and so on.
		*/
		for (int inode = 0; inode < N; ++inode) {
			if (node_colors[inode] == ic) {
				partition.push_back(inode);
			}
		}

		partitions.push_back(partition);
	}

	std::vector<int> indices;
	indices.push_back(0);
	int row = 0;
	for (Partition partition : partitions) {
		for (int variable : partition) {
			colors[row] = variable;
			row++;
		}
		indices.push_back(row);
	}

	return indices;
}


int main() {
	srand(13000);

	std::ifstream file("content/t2dal_a/t2dal_a.mtx");
  int num_row, num_col, num_lines, N;

  while (file.peek() == '%') file.ignore(2048, '\n');

  file >> num_row>> num_col >> num_lines;

  N = num_row;

	gpuMat* m = new gpuMat(N);
  std::fill(m->m, m->m + num_row * num_col, 0.0f);
  for (int l = 0; l < num_lines; l++)
  {
      float data;
      int row, col;
      file >> row >> col >> data;
      m->m[(row -1) + (col -1) * num_row] = data;
  }

  file.close();


	int nonZeros = 0;
	for (int i = 0; i < N; ++i) {
		for (int j = i; j < N; ++j) {
			if (fabs(m->m[i * N + j]) > EPS) {
				nonZeros++;
			}
		}
	}
	printf("GPU Gauss Seidel with Graph Coloring \npercent of non-zeros of M: %d%%\n", int(100.0f * float(nonZeros) / float(N * N)));

	gpuVec<float>* expected_solution = new gpuVec<float>(N);
	for (int i = 0; i < N; ++i) {
		expected_solution->v[i] = 8.0f * float(rand() % 100) / 100.0f - 4.0f;
	}

	gpuVec<float>* b = new gpuVec<float>(N);
	b = m->gpu_mult(expected_solution, b);

	/*
	With that, we have generated a linear system
	M*x = b.
	Now let's solve it!
	*/

	gpuVec<float>* x = new gpuVec<float>(N);
	for (int i = 0; i < N; ++i) {
		x->v[i] = 0.0f;
	}

	printf("solving linear system where N = %d", N);
	printf("\n");

	// graph coloring to partition the problem.
	gpuVec<int>* colors = new gpuVec<int>(N);
	auto started = std::chrono::high_resolution_clock::now();
	std::vector<int> h_indices = randomized_graph_coloring(m, colors->v, N);
	auto done = std::chrono::high_resolution_clock::now();

	std::cout << "Graph Coloring CPU time: " << std::chrono::duration_cast<std::chrono::milliseconds>(done - started).count() << "ms\n";
	printf("number of partitions: %zd\n", h_indices.size());

  delete m;
	delete x;
	delete b;
	delete expected_solution;
}

Writing cpucoloringgpugs.cu


In [ ]:
!nvcc cpucoloringgpugs.cu -o gpu1
!./gpu1

GPU Gauss Seidel with Graph Coloring 
percent of non-zeros of M: 0%
solving linear system where N = 4257
^C


In [ ]:
%%writefile csrcolortest.cu

#include <stdio.h>
#include <math.h>
#include <stdlib.h>
#include <chrono>
#include <iostream>
#include <set>
#include <vector>
#include <cusparse.h>
#include <cusparse_v2.h>
#include <algorithm>
#include <fstream>

const float EPS = 0.00001f;

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
	if (err != cudaSuccess) {
		fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
	}
	return err;
}

// vector of dimension Nx1
template <class T>
class gpuVec {
public:
	T* v;
  int N;

	__host__ gpuVec(int size) {
    N = size;
		cudaMallocManaged(&v, (T)(size * (int)sizeof(int)));
		for (int i = 0; i < N; ++i) {
			v[i] = 0;
		}
	}

	__host__ ~gpuVec()
	{
		cudaFree(v);
	}

	__host__ void print() {
		for (int i = 0; i < N; ++i) {
			printf("%f, ", v[i]);
		}
		printf("\n");
	}
};

// matrix of dimension NxN.
class gpuMat {
public:
	float* m;
  int N;

	__host__ gpuMat(int n) {
    N = n;
		cudaMallocManaged(&m, (float)(N * N * (int)sizeof(float)));
		for (int i = 0; i < N * N; ++i) {
			m[i] = 0.0f;
		}
	}

	__host__ ~gpuMat()
	{
		cudaFree(m);
	}

	__host__ void printdiag() {
		for (int i = 0; i < N; ++i) {
			{ printf("%f, ", m[i * N + i]); }
		}
		printf("\n");
	}

	__host__ void print() {
		for (int i = 0; i < N * N; ++i) {
			printf("%f, ", m[i]);
			if (i % N == 0 && i != 0) { printf("\n"); }
		}
		printf("\n");
	}

  // only used to get solution, b, from initial matrix, m.
  __host__ gpuVec<float>* gpu_mult(const gpuVec<float>* v, gpuVec<float>* r) const {

		for (int row = 0; row < N; ++row) {
			for (int col = 0; col < N; ++col) {
				r->v[row] += this->m[row * N + col] * v->v[col];
			}
		}
		return r;
	}
};

std::vector<int> color(int N, gpuMat* m, gpuVec<float>* x, gpuVec<float>* b, int* indices, int* max, int* dANnzPerRow,
	float* dCsrValA, int* dCsrRowPtrA, int* dCsrColIndA, int* totalANnz, cusparseHandle_t handle,
	cusparseMatDescr_t Adescr, float* dM, float* fractiontoColor, int* nrows, int* ncolors, int* coloring, int* reordering, cusparseColorInfo_t info)
{

  float* hCsrVal = (float*)malloc(*totalANnz * sizeof(float));
	int* hCsrRowPtr = (int*)malloc((N+1) * sizeof(int));
  int* hCsrColPtr = (int*)malloc(*totalANnz * sizeof(int));


	cusparseSdense2csr(handle, N, N, Adescr, dM, N, dANnzPerRow,
		dCsrValA, dCsrRowPtrA, dCsrColIndA);

	cusparseScsrcolor(handle, *nrows, *totalANnz, Adescr, dCsrValA, dCsrRowPtrA, dCsrColIndA, fractiontoColor, ncolors, coloring, reordering, info);

	printf("colors used: %d\nfraction to color: %f\n", *ncolors, *fractiontoColor);


  cudaMemcpy(hCsrVal, dCsrValA, *totalANnz * sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(hCsrRowPtr, dCsrRowPtrA, (N+1) * sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(hCsrColPtr, dCsrColIndA, *totalANnz * sizeof(int), cudaMemcpyDeviceToHost);


  int old = 0;
	*max = 0;
  for (int i = 0; i < (N+1); i++)
  {
		if (hCsrRowPtr[i] - old > *max)
		{
			*max = hCsrRowPtr[i] - old;
		}
  	old = hCsrRowPtr[i];
  }


	int* h_colors = (int*)malloc(N * sizeof(int));
	int* h_colindices = (int*)malloc(N * sizeof(int));
	cudaMemcpy(h_colors, coloring, N * sizeof(float), cudaMemcpyDeviceToHost);
	cudaMemcpy(h_colindices, reordering, N * sizeof(float), cudaMemcpyDeviceToHost);


	std::vector<std::pair<int, int>> new_order;
	for (int i = 0; i < N; i++)
	{
		new_order.push_back(std::make_pair(h_colors[i], h_colindices[i]));
	}
	std::sort(std::begin(new_order), std::end(new_order));


	std::vector<int> colors;
	int prev_color = -1;
	gpuMat* new_m = new gpuMat(N);
	gpuVec<float>* new_x = new gpuVec<float>(N); // reorder expected solution, not the x vector
	gpuVec<float>* new_b = new gpuVec<float>(N);

	for (int i = 0; i < N; i++)
	{
		if (prev_color != new_order[i].first)
		{
			colors.push_back(i);
			prev_color = new_order[i].first;
		}
		indices[i] = new_order[i].second;

		float* m_pt = m->m + (new_order[i].second * N);
		float* x_pt = x->v + (new_order[i].second);
		float* b_pt = b->v + (new_order[i].second);

		float* m_new_pt = new_m->m + (i * N);
		float* x_new_pt = new_x->v + (i);
		float* b_new_pt = new_b->v + (i);

		memcpy(m_new_pt, m_pt, N * (sizeof(float)));
		memcpy(x_new_pt, x_pt, (sizeof(float)));
		memcpy(b_new_pt, b_pt, (sizeof(float)));
	}

	m->m = new_m->m;
	x->v = new_x->v;
	b->v = new_b->v;

	return colors;
}

int main() {
	srand(0);

  std::ifstream file("content/flowmeter0/flowmeter0.mtx");
  int num_row, num_col, num_lines, N;

  while (file.peek() == '%') file.ignore(2048, '\n');

  file >> num_row>> num_col >> num_lines;

  N = num_row;

  gpuMat* m = new gpuMat(num_row);
  std::fill(m->m, m->m + num_row * num_col, 0.0f);

  for (int l = 0; l < num_lines; l++)
  {
      float data;
      int row, col;
      file >> row >> col >> data;
      m->m[(row -1) + (col -1) * num_row] = data;
  }

  file.close();

	int nonZeros = 0;
	for (int i = 0; i < N; ++i) {
		for (int j = i; j < N; ++j) {
			if (fabs(m->m[i * N + j]) > EPS) {
				nonZeros++;
			}
		}
	}
	printf("Gauss Seidel with Graph Coloring on the GPU using CuSparse\npercent of non-zeros of M: %d%%\n", int(100.0f * float(nonZeros) / float(N * N)));

	gpuVec<float>* expected_solution = new gpuVec<float>(N);
	for (int i = 0; i < N; ++i) {
		expected_solution->v[i] = 8.0f * float(rand() % 100) / 100.0f - 4.0f;
	}

	gpuVec<float>* b = new gpuVec<float>(N);
	b = m->gpu_mult(expected_solution, b);

	gpuVec<float>* x = new gpuVec<float>(N);
	for (int i = 0; i < N; ++i) {
		x->v[i] = 0.0f;
	}

	printf("solving linear system where N = %d", N);

	printf("\n");

  // Calculate Non-Zeros from original dense matrix
	int* nnz;
	cudaMallocManaged(&nnz, sizeof(int));
	*nnz = 0;

	for (int i = 0; i < N * N; i++)
	{
		if (fabs(m->m[i]) > 0.0f)
		{
			(*nnz)++;
		}
	}
	printf("nnz: %d\n", *nnz);

	int totalANnz;
	int* nrows;
	int* ncolors;
	int* coloring;
	int* iterations;
	int* reordering;
	int* dANnzPerRow;
	int* dCsrRowPtrA;
	int* dCsrColIndA;
	int* max = new int;
	float* dM;
	float* dCsrValA;
	float* fractiontoColor;
	std::vector<int> colors;
	gpuVec<int>* indices = new gpuVec<int>(N);
	gpuVec<float>* residual = new gpuVec<float>(N);

	cudaMalloc(&coloring, N * sizeof(float));
	cudaMalloc(&reordering, N * sizeof(float));
	cudaMalloc(&dM, N * N * sizeof(float));
	cudaMalloc((void**)&dANnzPerRow, sizeof(int) * N);
	cudaMallocManaged(&nrows, sizeof(int));
	cudaMallocManaged(&ncolors, sizeof(int));
	cudaMallocManaged(&iterations, sizeof(int));
	cudaMallocManaged(&fractiontoColor, sizeof(float));

	cusparseHandle_t handle = 0;
	cusparseCreate(&handle);
	cusparseColorInfo_t info;
	cusparseCreateColorInfo(&info);

	cusparseMatDescr_t Adescr = 0;
	cusparseCreateMatDescr(&Adescr);
	cusparseSetMatType(Adescr, CUSPARSE_MATRIX_TYPE_GENERAL);
	cusparseSetMatIndexBase(Adescr, CUSPARSE_INDEX_BASE_ZERO);

	cudaMemcpy(dM, m->m, N * N * sizeof(float), cudaMemcpyHostToDevice);

  // Total nnz, nnz per row
	cusparseSnnz(handle, CUSPARSE_DIRECTION_ROW, N, N, Adescr,
		dM, N, dANnzPerRow, &totalANnz);

	cudaMalloc((void**)&dCsrValA, sizeof(float) * totalANnz);
	cudaMalloc((void**)&dCsrRowPtrA, sizeof(int) * (N + 1));
	cudaMalloc((void**)&dCsrColIndA, sizeof(int) * totalANnz);

	*nrows = N;
	*ncolors = 0;
	*fractiontoColor = 0.15f;

	///////////////////////////////////////////////////// Start Processing

	auto started = std::chrono::high_resolution_clock::now();
	colors = color(N, m, expected_solution, b, indices->v, max, dANnzPerRow, dCsrValA, dCsrRowPtrA, dCsrColIndA, nnz, handle, Adescr, dM, fractiontoColor, nrows, ncolors, coloring, reordering, info);
	auto done = std::chrono::high_resolution_clock::now();

	std::cout << "Graph Coloring CPU time: " << std::chrono::duration_cast<std::chrono::milliseconds>(done - started).count() << "ms\n";

	printf("Max row length in CSR: %d\n", *max);
	printf("number of colors: %zd\n", colors.size());

  ///////////////////////////////////////////////////// End Processing

	delete m;
	delete x;
	delete b;
	delete residual;
	delete expected_solution;

	cudaFree(dANnzPerRow);
	cudaFree(dCsrValA);
	cudaFree(dCsrRowPtrA);
	cudaFree(dCsrColIndA);
	cudaFree(dM);;
	cudaFree(coloring);

	cusparseDestroyMatDescr(Adescr);
	cusparseDestroy(handle);

}

Overwriting csrcolortest.cu


In [ ]:
!nvcc csrcolortest.cu -lcusparse -o sol1
!./sol1

csrcolortest.cu: In function ‘std::vector<int> color(int, gpuMat*, gpuVec<float>*, gpuVec<float>*, int*, int*, int*, float*, int*, int*, int*, cusparseHandle_t, cusparseMatDescr_t, float*, float*, int*, int*, int*, int*, cusparseColorInfo_t)’:
csrcolortest.cu:107:96: warning: ‘cusparseStatus_t cusparseSdense2csr(cusparseHandle_t, int, int, cusparseMatDescr_t, const float*, int, const int*, float*, int*, int*)’ is deprecated: please use cusparseDenseToSparse instead [-Wdeprecated-declarations]
  107 |  cusparseSdense2csr(handle, N, N, Adescr, dM, N, dANnzPerRow,
      |                                                                                                ^
/usr/local/cuda/bin/../targets/x86_64-linux/include/cusparse.h:4104:1: note: declared here
 4104 | cusparseSdense2csr(cusparseHandle_t         handle,
      | ^~~~~~~~~~~~~~~~~~
csrcolortest.cu:107:96: warning: ‘cusparseStatus_t cusparseSdense2csr(cusparseHandle_t, int, int, cusparseMatDescr_t, const float*, int, const int*, 

### GPU Graph Coloring using CSRColor (CuSparse)
We use the coloring of CSRColor to reorder the initial dense matrix, and we apply the Gauss-Seidel kernel to it. Note that the value of N is twiddled up for the reduction kernel using: https://stackoverflow.com/questions/1322510/given-an-integer-how-do-i-find-the-next-largest-power-of-two-using-bit-twiddlin/1322548#1322548

In [ ]:
%%writefile finalgaussseidel.cu

#include <stdio.h>
#include <math.h>
#include <stdlib.h>
#include <chrono>
#include <iostream>
#include <set>
#include <vector>
#include <cusparse.h>
#include <cusparse_v2.h>
#include <algorithm>
#include <fstream>

const float EPS = 0.00001f;

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
	if (err != cudaSuccess) {
		fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
	}
	return err;
}

// vector of dimension Nx1
template <class T>
class gpuVec {
public:
	T* v;
  int N;

	__host__ gpuVec(int size) {
    N = size;
		cudaMallocManaged(&v, (T)(size * (int)sizeof(int)));
		for (int i = 0; i < N; ++i) {
			v[i] = 0;
		}
	}

	__host__ ~gpuVec()
	{
		cudaFree(v);
	}

	__host__ void print() {
		for (int i = 0; i < N; ++i) {
			printf("%f, ", v[i]);
		}
		printf("\n");
	}
};

// matrix of dimension NxN.
class gpuMat {
public:
	float* m;
  int N;

	__host__ gpuMat(int n) {
    N = n;
		cudaMallocManaged(&m, (float)(N * N * (int)sizeof(float)));
		for (int i = 0; i < N * N; ++i) {
			m[i] = 0.0f;
		}
	}

	__host__ ~gpuMat()
	{
		cudaFree(m);
	}

	__host__ void printdiag() {
		for (int i = 0; i < N; ++i) {
			{ printf("%f, ", m[i * N + i]); }
		}
		printf("\n");
	}

	__host__ void print() {
		for (int i = 0; i < N * N; ++i) {
			printf("%f, ", m[i]);
			if (i % N == 0 && i != 0) { printf("\n"); }
		}
		printf("\n");
	}

  // only used to get solution, b, from initial matrix, m.
  __host__ gpuVec<float>* gpu_mult(const gpuVec<float>* v, gpuVec<float>* r) const {

		for (int row = 0; row < N; ++row) {
			for (int col = 0; col < N; ++col) {
				r->v[row] += this->m[row * N + col] * v->v[col];
			}
		}
		return r;
	}
};


__global__ void gauss_seidel_partition(int N, float* x, const float* b, float* m, int* diags, int partition)
{
	extern __shared__ float cache[];

	int global_row = (blockIdx.x + partition);
  float omega = 1.0f;

	int tid = threadIdx.x;
	float temp = 0.0f;
	while (tid < N) {
			temp += m[tid + (N * global_row)] * x[tid];
			tid += blockDim.x;
	}

	cache[threadIdx.x] = temp;
	__syncthreads();

	int i = blockDim.x / 2;
	while (i != 0) {
		if (threadIdx.x < i)
			cache[threadIdx.x] += cache[threadIdx.x + i];
		__syncthreads();
		i /= 2;
	}

	if (threadIdx.x == 0)
	{
		int diag = diags[global_row];
		cache[0] -= m[diag + (N * global_row)] * x[diag];
		x[global_row] = (1.0f - omega)*x[global_row] + omega*(1.0f / m[diag + (N * global_row)]) * (b[global_row] - cache[0]);
	}
}

void gauss_seidel(int N, float* x, const float* b, float* m, int* colors, int ncolors, float* residual, int* iterations, int* indices) {
	int iter;
	float old_norm = 0.0f;
	int v = N;
	v--;
	v |= v >> 1;
	v |= v >> 2;
	v |= v >> 4;
	v |= v >> 8;
	v |= v >> 16;
	v++;
	int blockdim = std::min(v, 1024);
	int sharedsize = std::min(v, 1024)*sizeof(float);

	for (iter = 0; iter < 100; ++iter) {
		*iterations = iter;
		int prev_color = 0;
		for (int i = 1; i < ncolors; i += 1) {
			gauss_seidel_partition<<<colors[i] - prev_color, blockdim, sharedsize>>>(N, x, b, m, indices, colors[i - 1]);
			cudaDeviceSynchronize();
			prev_color = colors[i];
		}

		for (int row = 0; row < N; ++row) {
			for (int col = 0; col < N; ++col) {
				residual[row] += m[row * N + col] * x[col];
			}
		}
		float norm = 0.0f;
		float c = 0.0f;
		for (int i = 0; i < N; ++i) {
			float a = residual[i] - b[i];
			residual[i] = 0.0f;
			float y = a * a - c;
			float t = norm + y;
			c = (t - norm) - y;
			norm = t;
		}
		norm = sqrt(norm);
		if (fabs(norm - old_norm) < 0.0001f) {
			return;
		}
		old_norm = norm;
	}
}


__global__ void jacobi_partition(int N, float* x, const float* b, float* m, int* diags)
{
	extern __shared__ float cache[];

	int global_row = blockIdx.x;
  float omega = 1.0f;

	int tid = threadIdx.x;
	float temp = 0.0f;
	while (tid < N) {
			temp += m[tid + (N * global_row)] * x[tid];
			tid += blockDim.x;
	}

	cache[threadIdx.x] = temp;
	__syncthreads();

	int i = blockDim.x / 2;
	while (i != 0) {
		if (threadIdx.x < i)
			cache[threadIdx.x] += cache[threadIdx.x + i];
		__syncthreads();
		i /= 2;
	}

	if (threadIdx.x == 0)
	{
		int diag = diags[global_row];
		cache[0] -= m[diag + (N * global_row)] * x[diag];
		x[global_row] = (1.0f - omega)*x[global_row] + omega*(1.0f / m[diag + (N * global_row)]) * (b[global_row] - cache[0]);
	}
}

void jacobi(int N, float* x, const float* b, float* m, int* colors, int ncolors, float* residual, int* iterations, int* indices) {
	int iter;
	float old_norm = 0.0f;
	int v = N;
	v--;
	v |= v >> 1;
	v |= v >> 2;
	v |= v >> 4;
	v |= v >> 8;
	v |= v >> 16;
	v++;
	int blockdim = std::min(v, 1024);
	int sharedsize = std::min(v, 1024)*sizeof(float);

	for (iter = 0; iter < 100; ++iter) {
		*iterations = iter;
		jacobi_partition<<<N, blockdim, sharedsize>>>(N, x, b, m, indices);
		cudaDeviceSynchronize();


		for (int row = 0; row < N; ++row) {
			for (int col = 0; col < N; ++col) {
				residual[row] += m[row * N + col] * x[col];
			}
		}
		float norm = 0.0f;
		float c = 0.0f;
		for (int i = 0; i < N; ++i) {
			float a = residual[i] - b[i];
			residual[i] = 0.0f;
			float y = a * a - c;
			float t = norm + y;
			c = (t - norm) - y;
			norm = t;
		}
		norm = sqrt(norm);
		if (fabs(norm - old_norm) < 0.0001f) {
			return;
		}
		old_norm = norm;
	}
}



std::vector<int> color(int N, gpuMat* m, gpuVec<float>* x, gpuVec<float>* b, int* indices, int* max, int* dANnzPerRow,
	float* dCsrValA, int* dCsrRowPtrA, int* dCsrColIndA, int* totalANnz, cusparseHandle_t handle,
	cusparseMatDescr_t Adescr, float* dM, float* fractiontoColor, int* nrows, int* ncolors, int* coloring, int* reordering, cusparseColorInfo_t info)
{

  float* hCsrVal = (float*)malloc(*totalANnz * sizeof(float));
	int* hCsrRowPtr = (int*)malloc((N+1) * sizeof(int));
  int* hCsrColPtr = (int*)malloc(*totalANnz * sizeof(int));


	cusparseSdense2csr(handle, N, N, Adescr, dM, N, dANnzPerRow,
		dCsrValA, dCsrRowPtrA, dCsrColIndA);

	cusparseScsrcolor(handle, *nrows, *totalANnz, Adescr, dCsrValA, dCsrRowPtrA, dCsrColIndA, fractiontoColor, ncolors, coloring, reordering, info);

	printf("colors used: %d\nfraction to color: %f\n", *ncolors, *fractiontoColor);


  cudaMemcpy(hCsrVal, dCsrValA, *totalANnz * sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(hCsrRowPtr, dCsrRowPtrA, (N+1) * sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(hCsrColPtr, dCsrColIndA, *totalANnz * sizeof(int), cudaMemcpyDeviceToHost);


  int old = 0;
	*max = 0;
  for (int i = 0; i < (N+1); i++)
  {
		if (hCsrRowPtr[i] - old > *max)
		{
			*max = hCsrRowPtr[i] - old;
		}
  	old = hCsrRowPtr[i];
  }


	int* h_colors = (int*)malloc(N * sizeof(int));
	int* h_colindices = (int*)malloc(N * sizeof(int));
	cudaMemcpy(h_colors, coloring, N * sizeof(float), cudaMemcpyDeviceToHost);
	cudaMemcpy(h_colindices, reordering, N * sizeof(float), cudaMemcpyDeviceToHost);


	std::vector<std::pair<int, int>> new_order;
	for (int i = 0; i < N; i++)
	{
		new_order.push_back(std::make_pair(h_colors[i], h_colindices[i]));
	}
	std::sort(std::begin(new_order), std::end(new_order));


	std::vector<int> colors;
	int prev_color = -1;
	gpuMat* new_m = new gpuMat(N);
	gpuVec<float>* new_x = new gpuVec<float>(N); // reorder expected solution, not the x vector
	gpuVec<float>* new_b = new gpuVec<float>(N);

	for (int i = 0; i < N; i++)
	{
		if (prev_color != new_order[i].first)
		{
			colors.push_back(i);
			prev_color = new_order[i].first;
		}
		indices[i] = new_order[i].second;

		float* m_pt = m->m + (new_order[i].second * N);
		float* x_pt = x->v + (new_order[i].second);
		float* b_pt = b->v + (new_order[i].second);

		float* m_new_pt = new_m->m + (i * N);
		float* x_new_pt = new_x->v + (i);
		float* b_new_pt = new_b->v + (i);

		memcpy(m_new_pt, m_pt, N * (sizeof(float)));
		memcpy(x_new_pt, x_pt, (sizeof(float)));
		memcpy(b_new_pt, b_pt, (sizeof(float)));
	}

	m->m = new_m->m;
	x->v = new_x->v;
	b->v = new_b->v;

	return colors;
}

int main() {
	srand(0);

  std::ifstream file("content/flowmeter0/flowmeter0.mtx");
  int num_row, num_col, num_lines, N;

  while (file.peek() == '%') file.ignore(2048, '\n');

  file >> num_row>> num_col >> num_lines;

  N = num_row;

  gpuMat* m = new gpuMat(num_row);
  std::fill(m->m, m->m + num_row * num_col, 0.0f);

  for (int l = 0; l < num_lines; l++)
  {
      float data;
      int row, col;
      file >> row >> col >> data;
      m->m[(row -1) + (col -1) * num_row] = data;
  }

  file.close();

	int nonZeros = 0;
	for (int i = 0; i < N; ++i) {
		for (int j = i; j < N; ++j) {
			if (fabs(m->m[i * N + j]) > EPS) {
				nonZeros++;
			}
		}
	}
	printf("Gauss Seidel with Graph Coloring on the GPU using CuSparse\npercent of non-zeros of M: %d%%\n", int(100.0f * float(nonZeros) / float(N * N)));

	gpuVec<float>* expected_solution = new gpuVec<float>(N);
	for (int i = 0; i < N; ++i) {
		expected_solution->v[i] = 8.0f * float(rand() % 100) / 100.0f - 4.0f;
	}

	gpuVec<float>* b = new gpuVec<float>(N);
	b = m->gpu_mult(expected_solution, b);

	gpuVec<float>* x = new gpuVec<float>(N);
	for (int i = 0; i < N; ++i) {
		x->v[i] = 0.0f;
	}

	printf("solving linear system where N = %d", N);

	printf("\n");

  // Calculate Non-Zeros from original dense matrix
	int* nnz;
	cudaMallocManaged(&nnz, sizeof(int));
	*nnz = 0;

	for (int i = 0; i < N * N; i++)
	{
		if (fabs(m->m[i]) > 0.0f)
		{
			(*nnz)++;
		}
	}
	printf("nnz: %d\n", *nnz);

	int totalANnz;
	int* nrows;
	int* ncolors;
	int* coloring;
	int* iterations;
	int* reordering;
	int* dANnzPerRow;
	int* dCsrRowPtrA;
	int* dCsrColIndA;
	int* max = new int;
	float* dM;
	float* dCsrValA;
	float* fractiontoColor;
	std::vector<int> colors;
	gpuVec<int>* indices = new gpuVec<int>(N);
	gpuVec<float>* residual = new gpuVec<float>(N);

	cudaMalloc(&coloring, N * sizeof(float));
	cudaMalloc(&reordering, N * sizeof(float));
	cudaMalloc(&dM, N * N * sizeof(float));
	cudaMalloc((void**)&dANnzPerRow, sizeof(int) * N);
	cudaMallocManaged(&nrows, sizeof(int));
	cudaMallocManaged(&ncolors, sizeof(int));
	cudaMallocManaged(&iterations, sizeof(int));
	cudaMallocManaged(&fractiontoColor, sizeof(float));

	float gpu_total = 0.0f;
	cudaEvent_t start, stop;

	cusparseHandle_t handle = 0;
	cusparseCreate(&handle);
	cusparseColorInfo_t info;
	cusparseCreateColorInfo(&info);

	cusparseMatDescr_t Adescr = 0;
	cusparseCreateMatDescr(&Adescr);
	cusparseSetMatType(Adescr, CUSPARSE_MATRIX_TYPE_GENERAL);
	cusparseSetMatIndexBase(Adescr, CUSPARSE_INDEX_BASE_ZERO);

	cudaMemcpy(dM, m->m, N * N * sizeof(float), cudaMemcpyHostToDevice);

  // Total nnz, nnz per row
	cusparseSnnz(handle, CUSPARSE_DIRECTION_ROW, N, N, Adescr,
		dM, N, dANnzPerRow, &totalANnz);

	cudaMalloc((void**)&dCsrValA, sizeof(float) * totalANnz);
	cudaMalloc((void**)&dCsrRowPtrA, sizeof(int) * (N + 1));
	cudaMalloc((void**)&dCsrColIndA, sizeof(int) * totalANnz);

	*nrows = N;
	*ncolors = 0;
	*fractiontoColor = 1.0f;

	///////////////////////////////////////////////////// Start Processing

	auto started = std::chrono::high_resolution_clock::now();
	colors = color(N, m, expected_solution, b, indices->v, max, dANnzPerRow, dCsrValA, dCsrRowPtrA, dCsrColIndA, nnz, handle, Adescr, dM, fractiontoColor, nrows, ncolors, coloring, reordering, info);
	auto done = std::chrono::high_resolution_clock::now();

	std::cout << "Graph Coloring CPU time: " << std::chrono::duration_cast<std::chrono::milliseconds>(done - started).count() << "ms\n";

	printf("Max row length in CSR: %d\n", *max);

  // Copy coloring from CSRColor solution to GPU Gauss Seidel kernel
	gpuVec<int>* d_colors = new gpuVec<int>(colors.size());
	for (int i = 0; i < colors.size(); i++)
	{
		d_colors->v[i] = colors[i];
	}

	cudaEventCreate(&start);
  cudaEventCreate(&stop);

	cudaEventRecord(start);

	gauss_seidel(N, x->v, b->v, m->m, d_colors->v, colors.size(), residual->v, iterations, indices->v);
	//jacobi(N, x->v, b->v, m->m, d_colors->v, colors.size(), residual->v, iterations, indices->v);

	cudaEventRecord(stop);
	cudaEventSynchronize(stop);

	cudaEventElapsedTime(&gpu_total, start, stop);

	printf("Gauss Seidel GPU time: %fms\n", gpu_total);

	printf("number of partitions: %zd\n", colors.size());
	printf("number of iterations: %d\n", *iterations);

  ///////////////////////////////////////////////////// End Processing

	delete m;
	delete x;
	delete b;
	delete residual;
	delete expected_solution;

	cudaFree(dANnzPerRow);
	cudaFree(dCsrValA);
	cudaFree(dCsrRowPtrA);
	cudaFree(dCsrColIndA);
	cudaFree(dM);;
	cudaFree(coloring);

	cusparseDestroyMatDescr(Adescr);
	cusparseDestroy(handle);

}

Overwriting finalgaussseidel.cu


In [ ]:
!nvcc finalgaussseidel.cu -lcusparse -o sol2
!./sol2

finalgaussseidel.cu: In function ‘void gauss_seidel(int, float*, const float*, float*, int*, int, float*, int*, int*, int*, int)’:
finalgaussseidel.cu:145:79: warning: ‘cusparseStatus_t cusparseSdense2csr(cusparseHandle_t, int, int, cusparseMatDescr_t, const float*, int, const int*, float*, int*, int*)’ is deprecated: please use cusparseDenseToSparse instead [-Wdeprecated-declarations]
  145 |     cusparseSdense2csr(handle, len, N, Adescr, dM, len, dANnzPerRow,
      |                                                                               ^
/usr/local/cuda/bin/../targets/x86_64-linux/include/cusparse.h:4104:1: note: declared here
 4104 | cusparseSdense2csr(cusparseHandle_t         handle,
      | ^~~~~~~~~~~~~~~~~~
finalgaussseidel.cu:145:79: warning: ‘cusparseStatus_t cusparseSdense2csr(cusparseHandle_t, int, int, cusparseMatDescr_t, const float*, int, const int*, float*, int*, int*)’ is deprecated: please use cusparseDenseToSparse instead [-Wdeprecated-declarations]
  145 |   

### Decompose Martix of n colors into n Sparse Matrices, apply SpMV to update *x*
Create a CSR matrix for each reordered section of the colored matrix. Do SpMV for each color and the result is your solution in the range of the color. This is incomplete and not working correctly, likely due to some issue with my implementation of CuSparse SpMV. However, it is executing, so it is possible to time the performance.

In [ ]:
%%writefile finalgaussseidel.cu

#include <stdio.h>
#include <math.h>
#include <stdlib.h>
#include <chrono>
#include <iostream>
#include <set>
#include <vector>
#include <cusparse.h>
#include <cusparse_v2.h>
#include <algorithm>
#include <fstream>

const float EPS = 0.00001f;

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
	if (err != cudaSuccess) {
		fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
	}
	return err;
}

// vector of dimension Nx1
template <class T>
class gpuVec {
public:
	T* v;
  int N;

	__host__ gpuVec(int size) {
    N = size;
		cudaMallocManaged(&v, (T)(size * (int)sizeof(int)));
		for (int i = 0; i < N; ++i) {
			v[i] = 0;
		}
	}

	__host__ ~gpuVec()
	{
		cudaFree(v);
	}

	__host__ void print() {
		for (int i = 0; i < N; ++i) {
			printf("%f, ", v[i]);
		}
		printf("\n");
	}
};

// matrix of dimension NxN.
class gpuMat {
public:
	float* m;
  int N;

	__host__ gpuMat(int n) {
    N = n;
		cudaMallocManaged(&m, (float)(N * N * (int)sizeof(float)));
		for (int i = 0; i < N * N; ++i) {
			m[i] = 0.0f;
		}
	}

	__host__ ~gpuMat()
	{
		cudaFree(m);
	}

	__host__ void printdiag() {
		for (int i = 0; i < N; ++i) {
			{ printf("%f, ", m[i * N + i]); }
		}
		printf("\n");
	}

	__host__ void print() {
		for (int i = 0; i < N * N; ++i) {
			printf("%f, ", m[i]);
			if (i % N == 0 && i != 0) { printf("\n"); }
		}
		printf("\n");
	}

  // only used to get solution, b, from initial matrix, m.
  __host__ gpuVec<float>* gpu_mult(const gpuVec<float>* v, gpuVec<float>* r) const {

		for (int row = 0; row < N; ++row) {
			for (int col = 0; col < N; ++col) {
				r->v[row] += this->m[row * N + col] * v->v[col];
			}
		}
		return r;
	}
};


void gauss_seidel(int N, float* x, const float* b, float* m, int* colors, int ncolors,
float* residual, int* iterations, int* indices, int* dANnzPerRow, int nnz)
{
	int iter;
	float old_norm = 0.0f;

	float alpha = 1.0f;
	float beta = 0.0f;
  float* dX;
	cudaMallocManaged(&dX, N*sizeof(float));

	for (iter = 0; iter < 10; ++iter) {
		*iterations = iter;
		int global_row = 0;
		for (int i = 1; i < ncolors; i += 1) {

				int len = indices[i]-indices[i-1];
				global_row += len;
				//printf("len: %d\n", len);
				//--------------------------------------------------------------------------
				// CUSPARSE APIs
				cusparseHandle_t handle = 0;
				cusparseCreate(&handle);

				cusparseMatDescr_t Adescr = 0;
				cusparseCreateMatDescr(&Adescr);
				cusparseSetMatType(Adescr, CUSPARSE_MATRIX_TYPE_GENERAL);
				cusparseSetMatIndexBase(Adescr, CUSPARSE_INDEX_BASE_ZERO);

				float* dM;
				float* val;
				float* dY;
				int* row;
				int* col;
				int totalANnz;

				cudaMallocManaged(&dM, len*N*sizeof(float));
				cudaMemcpy(dM, m+global_row*N, len * N * sizeof(float), cudaMemcpyHostToDevice);

				cusparseSnnz(handle, CUSPARSE_DIRECTION_ROW, len, N, Adescr,
					dM, len, dANnzPerRow, &totalANnz);

				cudaMallocManaged(&val, totalANnz*sizeof(float));
				cudaMallocManaged(&col, totalANnz*sizeof(float));
				cudaMallocManaged(&row, (len+1)*sizeof(float));
				cudaMallocManaged(&dY, len*sizeof(float));

				cusparseSdense2csr(handle, len, N, Adescr, dM, len, dANnzPerRow,
					val, row, col);

				cusparseSpMatDescr_t matA;
				cusparseDnVecDescr_t vecX, vecY;
				void*                dBuffer    = NULL;
				size_t               bufferSize = 0;
				// Create sparse matrix A in CSR format
				cusparseCreateCsr(&matA, len, N, totalANnz,
																row, col, val,
																CUSPARSE_INDEX_32I, CUSPARSE_INDEX_32I,
																CUSPARSE_INDEX_BASE_ZERO, CUDA_R_32F);
				// Create dense vector X
				cusparseCreateDnVec(&vecX, N, dX, CUDA_R_32F);
				// Create dense vector Y
				cusparseCreateDnVec(&vecY, len, dY, CUDA_R_32F);
				// allocate an external buffer if needed
				cusparseSpMV_bufferSize(
																handle, CUSPARSE_OPERATION_NON_TRANSPOSE,
																&alpha, matA, vecX, &beta, vecY, CUDA_R_32F,
																CUSPARSE_SPMV_ALG_DEFAULT, &bufferSize);

				cudaMalloc(&dBuffer, bufferSize);
				// execute SpMV
				cusparseSpMV(handle, CUSPARSE_OPERATION_NON_TRANSPOSE,
                                 &alpha, matA, vecX, &beta, vecY, CUDA_R_32F,
                                 CUSPARSE_SPMV_ALG_DEFAULT, dBuffer);

				cudaMemcpy(&x[0]+colors[i], &dY[0]+colors[i], len*sizeof(float), cudaMemcpyDeviceToHost);


			cusparseDestroySpMat(matA);
			cusparseDestroyDnVec(vecX);
			cusparseDestroy(handle);
		}

		for (int row = 0; row < N; ++row) {
			for (int col = 0; col < N; ++col) {
				residual[row] += m[row * N + col] * x[col];
			}
		}
		float norm = 0.0f;
		float c = 0.0f;
		for (int i = 0; i < N; ++i) {
			float a = residual[i] - b[i];
			residual[i] = 0.0f;
			float y = a * a - c;
			float t = norm + y;
			c = (t - norm) - y;
			norm = t;
		}
		norm = sqrt(norm);
		if (fabs(norm - old_norm) < 0.0001f) {
			// destroy matrix/vector descriptors
			//return;
		}
		old_norm = norm;
	}
}


std::vector<int> color(int N, gpuMat* m, gpuVec<float>* x, gpuVec<float>* b, int* indices, int* max, int* dANnzPerRow,
	float* dCsrValA, int* dCsrRowPtrA, int* dCsrColIndA, int* totalANnz, cusparseHandle_t handle,
	cusparseMatDescr_t Adescr, float* dM, float* fractiontoColor, int* nrows, int* ncolors, int* coloring, int* reordering, cusparseColorInfo_t info)
{

  float* hCsrVal = (float*)malloc(*totalANnz * sizeof(float));
	int* hCsrRowPtr = (int*)malloc((N+1) * sizeof(int));
  int* hCsrColPtr = (int*)malloc(*totalANnz * sizeof(int));


	cusparseSdense2csr(handle, N, N, Adescr, dM, N, dANnzPerRow,
		dCsrValA, dCsrRowPtrA, dCsrColIndA);

	cusparseScsrcolor(handle, *nrows, *totalANnz, Adescr, dCsrValA, dCsrRowPtrA, dCsrColIndA, fractiontoColor, ncolors, coloring, reordering, info);

	printf("colors used: %d\nfraction to color: %f\n", *ncolors, *fractiontoColor);


  cudaMemcpy(hCsrVal, dCsrValA, *totalANnz * sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(hCsrRowPtr, dCsrRowPtrA, (N+1) * sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(hCsrColPtr, dCsrColIndA, *totalANnz * sizeof(int), cudaMemcpyDeviceToHost);


  int old = 0;
	*max = 0;
  for (int i = 0; i < (N+1); i++)
  {
		if (hCsrRowPtr[i] - old > *max)
		{
			*max = hCsrRowPtr[i] - old;
		}
  	old = hCsrRowPtr[i];
  }


	int* h_colors = (int*)malloc(N * sizeof(int));
	int* h_colindices = (int*)malloc(N * sizeof(int));
	cudaMemcpy(h_colors, coloring, N * sizeof(float), cudaMemcpyDeviceToHost);
	cudaMemcpy(h_colindices, reordering, N * sizeof(float), cudaMemcpyDeviceToHost);


	std::vector<std::pair<int, int>> new_order;
	for (int i = 0; i < N; i++)
	{
		new_order.push_back(std::make_pair(h_colors[i], h_colindices[i]));
	}
	std::sort(std::begin(new_order), std::end(new_order));


	std::vector<int> colors;
	int prev_color = -1;
	gpuMat* new_m = new gpuMat(N);
	gpuVec<float>* new_x = new gpuVec<float>(N); // reorder expected solution, not the x vector
	gpuVec<float>* new_b = new gpuVec<float>(N);

	for (int i = 0; i < N; i++)
	{
		if (prev_color != new_order[i].first)
		{
			colors.push_back(i);
			prev_color = new_order[i].first;
		}
		indices[i] = new_order[i].second;

		float* m_pt = m->m + (new_order[i].second * N);
		float* x_pt = x->v + (new_order[i].second);
		float* b_pt = b->v + (new_order[i].second);

		float* m_new_pt = new_m->m + (i * N);
		float* x_new_pt = new_x->v + (i);
		float* b_new_pt = new_b->v + (i);

		memcpy(m_new_pt, m_pt, N * (sizeof(float)));
		memcpy(x_new_pt, x_pt, (sizeof(float)));
		memcpy(b_new_pt, b_pt, (sizeof(float)));
	}

	m->m = new_m->m;
	x->v = new_x->v;
	b->v = new_b->v;



	return colors;
}

int main() {
	srand(0);

  std::ifstream file("content/flowmeter0/flowmeter0.mtx");
  int num_row, num_col, num_lines, N;

  while (file.peek() == '%') file.ignore(2048, '\n');

  file >> num_row>> num_col >> num_lines;

  N = num_row;

  gpuMat* m = new gpuMat(num_row);
  std::fill(m->m, m->m + num_row * num_col, 0.0f);

  for (int l = 0; l < num_lines; l++)
  {
      float data;
      int row, col;
      file >> row >> col >> data;
      m->m[(row -1) + (col -1) * num_row] = data;
  }

  file.close();

	int nonZeros = 0;
	for (int i = 0; i < N; ++i) {
		for (int j = i; j < N; ++j) {
			if (fabs(m->m[i * N + j]) > EPS) {
				nonZeros++;
			}
		}
	}
	printf("Gauss Seidel with Graph Coloring on the GPU using CuSparse\npercent of non-zeros of M: %d%%\n", int(100.0f * float(nonZeros) / float(N * N)));

	gpuVec<float>* expected_solution = new gpuVec<float>(N);
	for (int i = 0; i < N; ++i) {
		expected_solution->v[i] = 8.0f * float(rand() % 100) / 100.0f - 4.0f;
	}

	gpuVec<float>* b = new gpuVec<float>(N);
	b = m->gpu_mult(expected_solution, b);

	gpuVec<float>* x = new gpuVec<float>(N);
	for (int i = 0; i < N; ++i) {
		x->v[i] = 0.0f;
	}

	printf("solving linear system where N = %d", N);

	printf("\n");

  // Calculate Non-Zeros from original dense matrix
	int* nnz;
	cudaMallocManaged(&nnz, sizeof(int));
	*nnz = 0;

	for (int i = 0; i < N * N; i++)
	{
		if (fabs(m->m[i]) > 0.0f)
		{
			(*nnz)++;
		}
	}
	printf("nnz: %d\n", *nnz);

	int totalANnz;
	int* nrows;
	int* ncolors;
	int* coloring;
	int* iterations;
	int* reordering;
	int* dANnzPerRow;
	int* dCsrRowPtrA;
	int* dCsrColIndA;
	int* max = new int;
	float* dM;
	float* dCsrValA;
	float* fractiontoColor;
	std::vector<int> colors;
	gpuVec<int>* indices = new gpuVec<int>(N);
	gpuVec<float>* residual = new gpuVec<float>(N);

	cudaMalloc(&coloring, N * sizeof(float));
	cudaMalloc(&reordering, N * sizeof(float));
	cudaMalloc(&dM, N * N * sizeof(float));
	cudaMalloc((void**)&dANnzPerRow, sizeof(int) * N);
	cudaMallocManaged(&nrows, sizeof(int));
	cudaMallocManaged(&ncolors, sizeof(int));
	cudaMallocManaged(&iterations, sizeof(int));
	cudaMallocManaged(&fractiontoColor, sizeof(float));

	float gpu_total = 0.0f;
	cudaEvent_t start, stop;

	cusparseHandle_t handle = 0;
	cusparseCreate(&handle);
	cusparseColorInfo_t info;
	cusparseCreateColorInfo(&info);

	cusparseMatDescr_t Adescr = 0;
	cusparseCreateMatDescr(&Adescr);
	cusparseSetMatType(Adescr, CUSPARSE_MATRIX_TYPE_GENERAL);
	cusparseSetMatIndexBase(Adescr, CUSPARSE_INDEX_BASE_ZERO);

	cudaMemcpy(dM, m->m, N * N * sizeof(float), cudaMemcpyHostToDevice);

  // Total nnz, nnz per row
	cusparseSnnz(handle, CUSPARSE_DIRECTION_ROW, N, N, Adescr,
		dM, N, dANnzPerRow, &totalANnz);

	cudaMalloc((void**)&dCsrValA, sizeof(float) * totalANnz);
	cudaMalloc((void**)&dCsrRowPtrA, sizeof(int) * (N + 1));
	cudaMalloc((void**)&dCsrColIndA, sizeof(int) * totalANnz);

	*nrows = N;
	*ncolors = 0;
	*fractiontoColor = 1.0f;

	///////////////////////////////////////////////////// Start Processing

	auto started = std::chrono::high_resolution_clock::now();
	colors = color(N, m, expected_solution, b, indices->v, max, dANnzPerRow, dCsrValA, dCsrRowPtrA, dCsrColIndA, nnz, handle, Adescr, dM, fractiontoColor, nrows, ncolors, coloring, reordering, info);
	auto done = std::chrono::high_resolution_clock::now();

	std::cout << "Graph Coloring CPU time: " << std::chrono::duration_cast<std::chrono::milliseconds>(done - started).count() << "ms\n";

	printf("Max row length in CSR: %d\n", *max);

  // Copy coloring from CSRColor solution to GPU Gauss Seidel kernel
	gpuVec<int>* d_colors = new gpuVec<int>(colors.size());
	for (int i = 0; i < colors.size(); i++)
	{
		d_colors->v[i] = colors[i];
	}


	cudaEventCreate(&start);
  cudaEventCreate(&stop);

	cudaEventRecord(start);

	gauss_seidel(N, x->v, b->v, m->m, d_colors->v, colors.size(), residual->v, iterations, indices->v, dANnzPerRow, totalANnz);

	cudaEventRecord(stop);
	cudaEventSynchronize(stop);

	cudaEventElapsedTime(&gpu_total, start, stop);

	printf("Gauss Seidel GPU time: %fms\n", gpu_total);

	printf("number of partitions: %zd\n", colors.size());
	printf("number of iterations: %d\n", *iterations);

  ///////////////////////////////////////////////////// End Processing

	delete m;
	delete x;
	delete b;
	delete residual;
	delete expected_solution;

	cudaFree(dANnzPerRow);
	cudaFree(dCsrValA);
	cudaFree(dCsrRowPtrA);
	cudaFree(dCsrColIndA);
	cudaFree(dM);;
	cudaFree(coloring);

	cusparseDestroyMatDescr(Adescr);
	cusparseDestroy(handle);

}

Overwriting finalgaussseidel.cu


In [ ]:
!nvcc finalgaussseidel.cu -lcusparse -o sol2
!./sol2

finalgaussseidel.cu: In function ‘void gauss_seidel(int, float*, const float*, float*, int*, int, float*, int*, int*, int*, int)’:
finalgaussseidel.cu:145:79: warning: ‘cusparseStatus_t cusparseSdense2csr(cusparseHandle_t, int, int, cusparseMatDescr_t, const float*, int, const int*, float*, int*, int*)’ is deprecated: please use cusparseDenseToSparse instead [-Wdeprecated-declarations]
  145 |     cusparseSdense2csr(handle, len, N, Adescr, dM, len, dANnzPerRow,
      |                                                                               ^
/usr/local/cuda/bin/../targets/x86_64-linux/include/cusparse.h:4104:1: note: declared here
 4104 | cusparseSdense2csr(cusparseHandle_t         handle,
      | ^~~~~~~~~~~~~~~~~~
finalgaussseidel.cu:145:79: warning: ‘cusparseStatus_t cusparseSdense2csr(cusparseHandle_t, int, int, cusparseMatDescr_t, const float*, int, const int*, float*, int*, int*)’ is deprecated: please use cusparseDenseToSparse instead [-Wdeprecated-declarations]
  145 |   

### Notes & Future Work

- Under-relaxation was applied to select SuiteSparse matrices because the absence of an explicit 𝑏 vector prevented convergence; with extremely large coefficients in the matrix, iterative methods would otherwise diverge in the absence of a stabilizing factor
- Contribute to an open source simulation repository that may use Preconditioned Conjugate Gradient or Jacobi or serial Gauss Seidel, and use a multicoloring approach. Note that grid based approaches do not benefit from multicoloring, as at most there would be 2 colors, known as red-black coloring. Thus, this is suited to some physics simulations. While there may not be an inner product at every update, there may be a constraint update such as in Position Based Dynamics. Thus, a constraint graph must be generated to parallelize it.
- Use METIS library to partition into super nodes, color the super nodes and keep the processing of the super nodes, or color those individually. Note that this is only suitable for extremely large graphs with many SCC's. METIS is available to download with vcpkg. I have experimented with it but did not have time to get it working.
- Genetic algorithms on the GPU to create the coloring may also be worth exploring, especially if when initially testing it out, it outperforms CSRColor by producing less colors and a valid configuration.